In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv


## Import libraries and read in data

In [3]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from tqdm import tqdm

In [4]:
df = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_validation.csv')

In [5]:
sellp = pd.read_csv("../input/m5-forecasting-accuracy/sell_prices.csv")

In [6]:
cal = pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv")

In [7]:
cal["d"]=cal["d"].apply(lambda x: int(x.split("_")[1]))
sellp["id"] = sellp["item_id"] + "_" + sellp["store_id"] + "_validation"

In [8]:
cal[cal["d"]==1858]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1857,2016-02-29,11605,Monday,3,2,2016,1858,NaN,NaN,NaN,NaN,0,0,0


In [9]:
cal[cal["d"]==1886]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1885,2016-03-28,11609,Monday,3,3,2016,1886,NaN,NaN,NaN,NaN,0,0,0


In [10]:
for day in tqdm(range(1858, 1886)):
    wk_id = list(cal[cal["d"]==day]["wm_yr_wk"])[0]
    wk_price_df = sellp[sellp["wm_yr_wk"]==wk_id]
    df = df.merge(wk_price_df[["sell_price", "id"]], on=["id"], how='inner')
    df["unit_sales_" + str(day)] = df["sell_price"] * df["d_" + str(day)]
    df.drop(columns=["sell_price"], inplace=True)

100%|██████████| 28/28 [00:11<00:00,  2.36it/s]


In [11]:
df["dollar_sales"] = df[[c for c in df.columns if c.find("unit_sales")==0]].sum(axis=1)

In [12]:
df.drop(columns=[c for c in df.columns if c.find("unit_sales")==0], inplace=True)

In [13]:
df["weight"] = df["dollar_sales"] / df["dollar_sales"].sum()

In [14]:
df.drop(columns=["dollar_sales"], inplace=True)

In [15]:
display(cal[cal["d"]==1858])

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1857,2016-02-29,11605,Monday,3,2,2016,1858,NaN,NaN,NaN,NaN,0,0,0


In [16]:
display(cal[cal["d"]==1886])

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1885,2016-03-28,11609,Monday,3,3,2016,1886,NaN,NaN,NaN,NaN,0,0,0


## Use the naive logic to make forecasts for each of the aggregations level wise

- All 0s
- Average through all history
- Mean of previous 10, 20, 30, 60 days
- Same as previous 28 days
- Average of same day for all previous weeks

### Mean of history

In [17]:
df[[c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) <= 1885] +\
       ["id"]].set_index("id").transpose()

id,HOBBIES_1_001_CA_1_validation,HOBBIES_1_002_CA_1_validation,HOBBIES_1_003_CA_1_validation,HOBBIES_1_004_CA_1_validation,HOBBIES_1_005_CA_1_validation,HOBBIES_1_006_CA_1_validation,HOBBIES_1_007_CA_1_validation,HOBBIES_1_008_CA_1_validation,HOBBIES_1_009_CA_1_validation,HOBBIES_1_010_CA_1_validation,...,FOODS_3_818_WI_3_validation,FOODS_3_819_WI_3_validation,FOODS_3_820_WI_3_validation,FOODS_3_821_WI_3_validation,FOODS_3_822_WI_3_validation,FOODS_3_823_WI_3_validation,FOODS_3_824_WI_3_validation,FOODS_3_825_WI_3_validation,FOODS_3_826_WI_3_validation,FOODS_3_827_WI_3_validation
d_1,0,0,0,0,0,0,0,12,2,0,...,0,14,1,0,4,0,0,0,0,0
d_2,0,0,0,0,0,0,0,15,0,0,...,0,11,1,0,4,0,0,6,0,0
d_3,0,0,0,0,0,0,0,0,7,1,...,0,5,1,0,2,2,0,0,0,0
d_4,0,0,0,0,0,0,0,0,3,0,...,0,6,1,0,5,2,0,2,0,0
d_5,0,0,0,0,0,0,0,0,0,0,...,0,5,1,0,2,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_1881,2,0,0,3,1,0,0,10,0,0,...,2,4,1,0,2,0,0,0,2,5
d_1882,0,1,0,5,0,3,1,2,0,0,...,0,2,2,0,3,0,0,0,1,3
d_1883,1,1,1,0,0,1,0,0,0,0,...,6,2,3,0,1,0,0,2,1,2
d_1884,1,1,1,6,0,1,0,0,0,0,...,2,1,2,2,1,0,0,0,2,0


In [18]:
complete_historical_mean_df =\
    df[[c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) <= 1885] +\
       ["id"]].set_index("id").transpose().mean().reset_index()

In [19]:
complete_historical_mean_df.head()

,id,0
0,HOBBIES_1_001_CA_1_validation,0.303979
1,HOBBIES_1_002_CA_1_validation,0.260477
2,HOBBIES_1_003_CA_1_validation,0.144297
3,HOBBIES_1_004_CA_1_validation,1.717241
4,HOBBIES_1_005_CA_1_validation,0.960743


In [20]:
df[[c for c in df.columns if c.find("d_")==0]].sum(axis=1).min()

10

In [21]:
def find_first_non_0(s):
    assert type(s) == np.ndarray
    return (s!=0).argmax(axis=0)

In [22]:
non_0_strt_arr = []
hist_arr = np.array(df[[c for c in df.columns if c.find("d_")==0]])
for i in tqdm(range(len(df))):
    non_0_strt_arr.append(find_first_non_0(hist_arr[i, :]))

100%|██████████| 30490/30490 [00:00<00:00, 90914.11it/s]


In [23]:
df.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913,weight
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,3,0,1,1,1,3,0,1,1,0.000063


In [24]:
test = list(df[[c for c in df.columns if c.find("d_")==0] +\
                ["id"]].set_index("id").transpose()["HOBBIES_1_001_CA_1_validation"])

In [25]:
print("Supposedly first non-zero value equals:", test[non_0_strt_arr[0]], 
      "on the", non_0_strt_arr[0], "day",
     "\nSum of all values before the supposedly first non-zero value is:", 
     sum(test[: non_0_strt_arr[0]]),
     "\nSum of all values after the supposedly first non-zero value is:", 
     sum(test[non_0_strt_arr[0]:]))

Supposedly first non-zero value equals: 1 on the 901 day 
Sum of all values before the supposedly first non-zero value is: 0 
Sum of all values after the supposedly first non-zero value is: 600


In [26]:
num_non_zero = 1885 - np.array(non_0_strt_arr)

In [27]:
non_zero_historical_mean_arr = np.array(df[[c for c in df.columns if c.find("d_")==0 and int(c.split("_")[1]) <= 1885] +\
   ["id"]].set_index("id").transpose().sum().reset_index()[0]) / num_non_zero

In [28]:
# days 1886 to 1913 are local test weeks
for d in range(1, 29):
    df["F_1_" + str(1885+d)] = list(complete_historical_mean_df[0])
    df["F_2_" + str(1885+d)] = non_zero_historical_mean_arr

In [29]:
method_dict = {1: "complete historical mean", 2: "historical mean after first non-zero"}

### Mean of recent x days

In [30]:
num_non_zero.min()

40

In [31]:
historical_mean_df10 =\
    df[[c for c in df.columns if c.find("d_")==0 and\
        int(c.split("_")[1]) in range(1876, 1886)] +\
       ["id"]].set_index("id").transpose().mean().reset_index()

historical_mean_df20 =\
    df[[c for c in df.columns if c.find("d_")==0 and\
        int(c.split("_")[1]) in range(1866, 1886)] +\
       ["id"]].set_index("id").transpose().mean().reset_index()

historical_mean_df30 =\
    df[[c for c in df.columns if c.find("d_")==0 and\
        int(c.split("_")[1]) in range(1856, 1886)] +\
       ["id"]].set_index("id").transpose().mean().reset_index()

historical_mean_df40 =\
    df[[c for c in df.columns if c.find("d_")==0 and\
        int(c.split("_")[1]) in range(1846, 1886)] +\
       ["id"]].set_index("id").transpose().mean().reset_index()

In [32]:
# days 1886 to 1913 are local test weeks
for d in range(1, 29):
    df["F_3_" + str(1885+d)] = list(historical_mean_df10[0])
    df["F_4_" + str(1885+d)] = list(historical_mean_df20[0])
    df["F_5_" + str(1885+d)] = list(historical_mean_df30[0])
    df["F_6_" + str(1885+d)] = list(historical_mean_df40[0])

In [33]:
method_dict[3] = "historical mean of recent 10 days"
method_dict[4] = "historical mean of recent 20 days"
method_dict[5] = "historical mean of recent 30 days"
method_dict[6] = "historical mean of recent 40 days"

### Same as previous 28 days

In [34]:
for d in range(1, 29):
    df["F_7_" + str(1885 + d)] = df["d_" + str(1885 + d - 28)]

In [35]:
method_dict[7] = "same as last 28 days"

### Historical averages from same day of the year

In [37]:
display(cal[cal["d"]==1886])
display(cal[cal["d"]==1886 - 364])

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1885,2016-03-28,11609,Monday,3,3,2016,1886,NaN,NaN,NaN,NaN,0,0,0


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
1521,2015-03-30,11509,Monday,3,3,2015,1522,NaN,NaN,NaN,NaN,0,0,0


In [38]:
denominator = [(num // 364) if (num // 364) > 0 else 1 for num in num_non_zero]
for d in range(1, 29): 
    df["F_8_" + str(1885 + d)] = (df["d_" + str(1885 + d - 364*1)]+\
                                  df["d_" + str(1885 + d - 364*2)]+\
                                  df["d_" + str(1885 + d - 364*3)]+\
                                  df["d_" + str(1885 + d - 364*4)]+\
                     df["d_" + str(1885 + d - 364*5)]) / denominator

In [39]:
method_dict[8] = "average of same day in historical years"

## Predicting forecast, ground truth values, and weights for all the higher level series by aggregating

In [40]:
agg_df = pd.DataFrame(df[[c for c in df.columns if c.find("d_") == 0 or c.find("F_") == 0]].sum()).transpose()
agg_df["level"] = 1
agg_df["weight"] = 1/12
column_order = agg_df.columns

In [41]:
agg_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,F_8_1906,F_8_1907,F_8_1908,F_8_1909,F_8_1910,F_8_1911,F_8_1912,F_8_1913,level,weight
0,32631.0,31749.0,23783.0,25412.0,19146.0,29211.0,28010.0,37932.0,32736.0,25572.0,...,45755.466667,35363.383333,32867.816667,32691.55,32935.033333,37391.55,46296.35,47663.15,1,0.083333


In [42]:
level_groupings = {2: ["state_id"], 3: ["store_id"], 4: ["cat_id"], 5: ["dept_id"], 
              6: ["state_id", "cat_id"], 7: ["state_id", "dept_id"], 8: ["store_id", "cat_id"], 9: ["store_id", "dept_id"],
              10: ["item_id"], 11: ["item_id", "state_id"]}

In [43]:
for level in tqdm(level_groupings):
    temp_df = df.groupby(by=level_groupings[level]).sum().reset_index(drop=True)
    temp_df["level"] = level
    temp_df["weight"] /= 12
    agg_df = agg_df.append(temp_df[column_order])

del temp_df

100%|██████████| 10/10 [00:08<00:00,  1.19it/s]


In [44]:
df["weight"] /= 12

In [45]:
print(df.shape[0], agg_df.shape[0], df.shape[0] + agg_df.shape[0])

30490 12350 42840


In [46]:
agg_df["weight"].sum() + df["weight"].sum()

0.9999999999999996

## Calculating RMSSE for all series using the equation

In [47]:
h = 28
n = 1885
def rmsse(ground_truth, forecast, train_series, axis=1):
    # assuming input are numpy array or matrices
    assert axis == 0 or axis == 1
    assert type(ground_truth) == np.ndarray and type(forecast) == np.ndarray and type(train_series) == np.ndarray
    
    if axis == 1:
        # using axis == 1 we must guarantee these are matrices and not arrays
        assert ground_truth.shape[1] > 1 and forecast.shape[1] > 1 and train_series.shape[1] > 1
    
    numerator = ((ground_truth - forecast)**2).sum(axis=axis)
    if axis == 1:
        denominator = 1/(n-1) * ((train_series[:, 1:] - train_series[:, :-1]) ** 2).sum(axis=axis)
    else:
        denominator = 1/(n-1) * ((train_series[1:] - train_series[:-1]) ** 2).sum(axis=axis)
    return (1/h * numerator/denominator) ** 0.5

In [48]:
train_series_cols = [c for c in df.columns if c.find("d_") == 0][:-28]
ground_truth_cols = [c for c in df.columns if c.find("d_") == 0][-28:]

forecast_cols_dict = {}
for i in range(1, 9):
    forecast_cols_dict[i] = [c for c in df.columns if c.find("F_"+str(i)+"_") == 0]

In [49]:
for i in range(1, 9):
    df["rmsse_" + str(i)] = rmsse(np.array(df[ground_truth_cols]), 
        np.array(df[forecast_cols_dict[i]]), np.array(df[train_series_cols]))
    agg_df["rmsse_" + str(i)] = rmsse(np.array(agg_df[ground_truth_cols]), 
        np.array(agg_df[forecast_cols_dict[i]]), np.array(agg_df[train_series_cols]))

In [50]:
for i in range(1, 9):
    df["wrmsse_" + str(i)] = df["weight"] * df["rmsse_" + str(i)]
    agg_df["wrmsse_" + str(i)] = agg_df["weight"] * agg_df["rmsse_" + str(i)]

In [51]:
for i in range(1, 9):
    print("method:", method_dict[i])
    print(df["wrmsse_" + str(i)].sum() + agg_df["wrmsse_" + str(i)].sum())
    print()

method: complete historical mean
1.654145319495275

method: historical mean after first non-zero
1.2467448505459677

method: historical mean of recent 10 days
1.1682107851100303

method: historical mean of recent 20 days
1.1412136208411774

method: historical mean of recent 30 days
1.1348927688382013

method: historical mean of recent 40 days
1.1380883217322861

method: same as last 28 days
0.8558233199674036

method: average of same day in historical years
0.9714971126875971



## Saving predictions in the submission file

In [52]:
sample_sub = pd.read_csv("../input/m5-forecasting-accuracy/sample_submission.csv")

In [53]:
sample_sub.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [54]:
sample_sub.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60979,FOODS_3_827_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
(sample_sub["id"][:len(df)] == df["id"]).all()

True

In [56]:
submit_df = df[["id"]]
for i in range(1, 29):
    submit_df["F" + str(i)] = df["F_7_" + str(1885 + i)]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [57]:
submit_df2 = submit_df.copy()
submit_df2["id"] = submit_df2["id"].apply(lambda x: x.replace('validation',
                                                              'evaluation'))

In [58]:
submit_df = submit_df.append(submit_df2).reset_index(drop=True)

In [ ]:
submit_df.to_csv("baseline_submission.csv", index=False)